In [27]:
"""ch1_getting_starting: vector_add"""

'ch1_getting_starting: vector_add'

In [16]:
# %load vector_add.py
# improt package
import tvm
from tvm import te # te stands for tensor expression
import time
import timeit
import numpy as np
from matplotlib import pyplot as plt
from IPython import display
try:
    import torch 
    print("\"import torch\" success...")
except:
    pass


"import torch" success...


In [17]:
# implemention with numpy
def get_abc(shape, constructor = None):
    """ return random a, b and empty c with the same shape
    """
    np.random.seed(0)
    a = np.random.normal(size = shape).astype(np.float32)
    b = np.random.normal(size = shape).astype(np.float32)
    c = np.empty(shape = shape, dtype = np.float32)
    if constructor is not None:
        a, b, c = [constructor(i) for i in (a, b, c)]
    return a, b, c

n = 100
a, b, c = get_abc(n, None)
c = a + b
def vector_add_normal(n, a, b):
    d = np.empty_like(a)
    for i in range(n):
        d[i] = a[i] + b[i]
    return d
d = vector_add_normal(n, a, b)
np.testing.assert_equal(c, d)

In [18]:
# defining the tvm computation
def vector_add(n):
    """TVM expressoion for vector add"""
    A = te.placeholder((n,), name = 'a')
    B = te.placeholder((n,), name = 'b')
    C = te.compute(A.shape, lambda i : A[i] + B[i], name = 'c')
    return A, B, C

A, B, C = vector_add(n)
print(type(A), type(C))
print(A.dtype,  A.shape)
print(type(A.op), type(C.op))
print(A.op.__class__.__bases__[0])

# create a schedule: execute plan
s = te.create_schedule(C.op)
print(type(s), type(s[C.op]))
print(tvm.lower(s, [A, B, C], simple_mode = True))

<class 'tvm.te.tensor.Tensor'> <class 'tvm.te.tensor.Tensor'>
float32 [100]
<class 'tvm.te.tensor.PlaceholderOp'> <class 'tvm.te.tensor.ComputeOp'>
<class 'tvm.te.tensor.Operation'>
<class 'tvm.te.schedule.Schedule'> <class 'tvm.te.schedule.Stage'>
@main = primfn(a_1: handle, b_1: handle, c_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(float32), float32, [100], []),
             b: Buffer(b_2: Pointer(float32), float32, [100], []),
             c: Buffer(c_2: Pointer(float32), float32, [100], [])}
  buffer_map = {a_1: a, b_1: b, c_1: c}
  preflattened_buffer_map = {a_1: a_3: Buffer(a_2, float32, [100], []), b_1: b_3: Buffer(b_2, float32, [100], []), c_1: c_3: Buffer(c_2, float32, [100], [])} {
  for (i: int32, 0, 100) {
    c[i] = (a[i] + b[i])
  }
}




In [19]:
# compilation and execution
mod = tvm.build(s, [A, B, C])
print(type(mod))
""" tvm.ndarray.NDArray ---> np.ndarray (.asnumpy())
    np.ndarray ---> tvm.ndarray.NDArray (tvm.nd.array())
"""
a, b, c = get_abc(n, tvm.nd.array)
mod(a, b, c)
np.testing.assert_equal(a.asnumpy() + b.asnumpy(), c.asnumpy())

<class 'tvm.driver.build_module.OperatorModule'>


In [20]:
# argument constraints-1
try:
    """ TVM will check if the input shapes satisfy this specification.
        A/B/C's shape is 100, a/b/c's shape is 200. shape not satisfy!
    """
    a, b, c = get_abc(200, tvm.nd.array)
    mod(a, b, c)
except tvm.TVMError as e:
    print(e)
  

Traceback (most recent call last):
  1: TVMFuncCall
  0: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::WrapPackedFunc(int (*)(TVMValue*, int*, int, TVMValue*, int*, void*), tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  File "/staff/qiaoliang/ACSA科研项目/TVM/tvm/src/runtime/library_module.cc", line 80
TVMError: 
---------------------------------------------------------------
An error occurred during the execution of TVM.
For more information, please see: https://tvm.apache.org/docs/errors.html
---------------------------------------------------------------

  Check failed: ret == 0 (-1 vs. 0) : Assert fail: (100 == int32(arg.a.shape[0])), Argument arg.a.shape[0] has an unsatisfied constraint: (100 == int32(arg.a.shape[0]))


In [21]:
# argument constraints-1
try:
    """ An error will appear if input with a different data type.
        A/B/C's dtype is float32, a/b/c's dtype is float64. dtype not satisfy!
    """
    a, b, c = get_abc(100, tvm.nd.array)
    a, b, c = [tvm.nd.array(i.asnumpy().astype(np.float64)) for i in (a, b, c)]
    mod(a, b, c)
except tvm.TVMError as e:
    print(e)

Traceback (most recent call last):
  1: TVMFuncCall
  0: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::WrapPackedFunc(int (*)(TVMValue*, int*, int, TVMValue*, int*, void*), tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  File "/staff/qiaoliang/ACSA科研项目/TVM/tvm/src/runtime/library_module.cc", line 80
TVMError: 
---------------------------------------------------------------
An error occurred during the execution of TVM.
For more information, please see: https://tvm.apache.org/docs/errors.html
---------------------------------------------------------------

  Check failed: ret == 0 (-1 vs. 0) : Assert fail: (((tir.tvm_struct_get(arg.a, 0, 5) == (uint8)2) && (tir.tvm_struct_get(arg.a, 0, 6) == (uint8)32)) && (tir.tvm_struct_get(arg.a, 0, 7) == (uint16)1)), arg.a.dtype is expected to be f

In [22]:
# saving and loading a module
mod_fname = 'vector-add.tar'
mod.export_library(mod_fname)
loaded_mod = tvm.runtime.load_module(mod_fname)

# verify the results
a, b, c = get_abc(100, tvm.nd.array)
loaded_mod(a, b, c)
np.testing.assert_equal(a.asnumpy() + b.asnumpy(), c.asnumpy())

In [25]:
# summary
"""
    Implementing an operator using TVM has 3 steps:
        1.Computation: Declare the computation by specifying input and output shapes and how each output element is computed.
        2.Schedule: Create a schedule to fully utilize the machine resources.
        3.Compile: Compile to the hardware target.
"""

'\n    Implementing an operator using TVM has 3 steps:\n        1.Computation: Declare the computation by specifying input and output shapes and how each output element is computed.\n        2.Schedule: Create a schedule to fully utilize the machine resources.\n        3.Compile: Compile to the hardware target.\n'